In [1]:
from random import seed
from random import random
import numpy as np
 
# 네트워크 초기 설정
def initialize_network(n_inputs, n_hidden, n_outputs): 
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)] #입력 노드의 개수 + bias 만큼 가중치를 생성함. 
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)] #은닉 노드의 개수 + bias 만큼 가중치를 생성함. 
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2) 
# 입력노드는 2개, 은닉노드는 1개, 부류는 두 개로 초기 네트워크를 설정함.
# hidden_layer : 입력노드의 수는 2개 이기 때문에 총 세 개 생성 (bias, x1, x2의 가중치), 은닉노드는 한 개 이기 때문에 한 세트 생성. 
# output_layer : 은닉노드의 수는 1개 이기 때문에 총 두 개 생성 (bias, h1), 클래스는 두 개이기 때문에 가중치가 각 클래스에 따른 두 세트로 생성됨.

#초기 설정한 네트워크를 확인함.
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [2]:
#순전파 함수
def activate(weights, inputs): #각각의 입력값과 가중치를 곱해서 더해주는 함수
    activation = weights[-1] #bias의 가중치는 곱해줄 게 없기 때문에 초기에 먼저 더해줌.
    for i in range(len(weights)-1): #bias를 제외한 각각의 입력노드 요소에 접근
        activation += weights[i] * inputs[i]  # 가중치와 입력값을 곱해서 activation에 더해준다.
    return activation #그걸 반환해준다 !! 

def sigmoid(activation):
    return 1.0 / (1.0 + np.exp(-activation)) # 시그모이드 구현

def forward_propagate(network, row): 
    inputs = row 
    for layer in network: #입력노드의 가중치부터 은닉노드의 가중치까지 순차적으로 접근
        new_inputs = [] 
        for neuron in layer: #해당 노드의 가중치에 각각 접근
            activation = activate(neuron['weights'], inputs) #해당 노드의 가중치를 activate 함수의 결과로 반환받음.
            neuron['output'] = sigmoid(activation) # 나온 계산 값을 그대로 쓰나요? ==> 아니요! 활성함수인 시그모이드에 넣었어용 ^--^
            new_inputs.append(neuron['output']) # new_input은 다음 히든층에 들어갈 값이죠? ==> 네 맞아요 input에 대한 출력값이고 이는 즉 다음 레이어의 입력값이 됩니다.
        inputs = new_inputs #다음 노드의 입력값이 되도록 inputs 값을 업데이트 해줍니다. 
    return inputs #그걸 반환해준다!

**여기까지는 순전파 학습과정이었습니다. 이 과정이 끝나면 가중치가 바뀌나요?  
답변을 답변의 근거 코딩 결과와 함께 보여주세요.**

In [3]:
row = [1, 0, None]
output = forward_propagate(network, row)
print(output) #부류가 2개였으니까 각 부류에 대한 outputs값이 두 개 나옵니다. 0과 1 이렇게만 나오는 줄 알았는데 시그모이드가 연속형이라 이런 값이 나오나봐요!!

[0.6629970129852887, 0.7253160725279748]


In [4]:
def sigmoid_derivative(output):
    return output * (1.0 - output) # 시그모이드 미분 (y(1-y))

def backward_propagate_error(network, expected):
    
    for i in reversed(range(len(network))): #reversed이기 때문에 0, 1 순서로 실행되는 것이 아니라 1, 0순서로 실행, 첫 실행의 i는 1이다.
        layer = network[i]
        errors = []
        if i != len(network)-1: 
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error) 
        else: #이거부터 실행됨. 역전이니까!
            for j in range(len(layer)):
                neuron = layer[j] 
                errors.append(expected[j] - neuron['output']) # 역전파시 오차는 어떻게 설정했나요? ==> 실제값에서 출력값(output)를 뺀 값으로 설정했습니다!
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] =  errors[j] * sigmoid_derivative(neuron['output'])


In [5]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002711797799238243}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14813473120687762}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.05472601157879688}]


In [6]:
def weights_update(network, row, l_rate): #가중치를 업데이트하자!!
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta'] # 퍼셉트론 학습 규칙
            

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row) # 순전파 
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) # 예측값의 오차 합
            backward_propagate_error(network, expected) #구한 expected 값을 이용해 역전파를 수행
            weights_update(network, row, l_rate) 
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [7]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [8]:
list(range(20))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [9]:
n_inputs = 2 # 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는? ==> 입력노드는 feature 개수입니다!! 2입니다!!
n_outputs = 2 # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠? ==> 부류의 개수입니다!! 여기서는 0과 1이니까 2입니다!!
network = initialize_network(n_inputs, 2, n_outputs)

for layer in network:
    print(layer)
    
train_network(network, dataset, 0.5, 20, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.

# 학습된(최적화)된 네트워크가 초기 네트워크와 달리 어떻게 변하였는지 출력하시오. (layer별로, hint : for문))

for layer in network:
    print(layer)


[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]}]
[{'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349]}, {'weights': [0.02834747652200631, 0.8357651039198697, 0.43276706790505337]}]
>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrate=0.500

In [10]:
# 학습한 네트워크로 예측값을 뽑아보자.
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs)) 
# 순전파 결과에서 어떤것이 최종 아웃풋이 되나요? 
# ==> outputs은 활성함수(시그모이드)를 통해 나온 값으로 0일 때, 1일 때로 나누어져 있습니다 
# 그리고 그 중 더 값이 큰 쪽의 인덱스를 반환하여 그 값을 최종 아웃풋으로 사용하는 것 같습니다!!

In [11]:
# 네트워크가 잘 학습되었는지 확인해보자. 

for row in dataset:
    prediction = predict(network, row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    print('실제값=%d, 예측값=%d' % (row[-1], prediction))

    #예측이 Perfect하다!!


실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1


 제가 수식적인 부분을 완전히 이해하지 못해서 주석을 많이 못달았어요 ㅠㅠ  
 
그래도 강의 자료 올려주신 것과 추가로 인강도 들어가며 뉴럴 네트워크 많이 공부하려고 했다는 점만 알아주세요,,,  

험난한 이 시국에 코로나 조심하시구요 쉬는 동안 더 공부해가겠습니다!!  

앙상블 때부터 좋은 강의 해주셔서 너무 감사해요 과제하면서 많이 배워갑니다!! :>>> ♡
